In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1801a438-441c-80b8-e64b-6b7ab5b0d6d9)
GPU 1: Tesla T4 (UUID: GPU-4819fa2d-c783-bdcc-96ae-13a7a1f101d6)


In [6]:
%%time
!pip install -q -U langchain tiktoken pypdf chromadb faiss-gpu
!pip install -q -U transformers InstructorEmbedding sentence_transformers
!pip install -q -U accelerate bitsandbytes xformers einops

CPU times: user 510 ms, sys: 104 ms, total: 614 ms
Wall time: 37.4 s


In [ ]:
!jupyter labextension install jupyter-leaflet # to display widgets correctly

In [86]:
!cp -r ../input/hp-embeddings-instructor-base-800-0/ ./

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Importing

In [64]:
import warnings
warnings.filterwarnings('ignore')

import os
import glob
import textwrap
import time

import langchain

#loaders
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

# splits
from langchain import PromptTemplate, ConversationChain, LLMChain

# vector stores
from langchain.vectorstores import Chroma, FAISS

# models
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

# retrievers
from langchain.chains import RetrievalQA, ConversationalRetrievalChain

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [14]:
os.listdir('../input')

['hp-embeddings-instructor-base-800-0', 'harry-potter-books-in-pdf-1-7']

In [40]:
os.listdir()

['hp-embeddings-instructor-base-800-0', '.virtual_documents']

In [15]:
glob.glob('../input/harry-potter-books-in-pdf-1-7/HP books/*')

['../input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 1 - The Sorcerers Stone.pdf',
 '../input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 5 - The Order of the Phoenix.pdf',
 '../input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 4 - The Goblet of Fire.pdf',
 '../input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 3 - The Prisoner of Azkaban.pdf',
 '../input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 6 - The Half-Blood Prince.pdf',
 '../input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 7 - The Deathly Hallows.pdf',
 '../input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 2 - The Chamber of Secrets.pdf']

In [80]:
class CFG:
    # LLMs - wizardlm, bloom, falcon, llama2-7b, llama2-13b
    model_name = 'llama2-7b'
    temperature = 0
    top_p = 0.95
    repitition_penalty = 1.15
    
    # splitting
    split_chunk_size = 800
    split_overlap = 0
    
    #embeddings
    embeddings_model_repo = 'hkunlp/instructor-base'
    
    # similar passages
    k = 3
    
    # paths
    PDFs_path = '../input/harry-potter-books-in-pdf-1-7/HP books/'
    embeddings_path = '/kaggle/working/hp-embeddings-instructor-base-800-0/harry-potter-vectordb-chroma'
    persist_dir = './harry-potter-vectordb-chroma'

### Model

In [17]:
model_name = 'llama2-7b'
model_repo = 'daryl149/llama-2-7b-chat-hf'

tokenizer = AutoTokenizer.from_pretrained(model_repo,
                                          use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_repo,
    load_in_4bit = True,
    device_map = 'auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
)

max_len = 2048

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
pipe = pipeline(
    task ='text-generation',
    model = model,
    tokenizer = tokenizer,
    pad_token_id = tokenizer.eos_token_id,
    max_length = max_len,
    temperature = CFG.temperature,
    top_p = CFG.top_p,
    repetition_penalty = CFG.repitition_penalty
)

llm = HuggingFacePipeline(pipeline = pipe)

In [27]:
llm

HuggingFacePipeline(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f8b80a4fe80>, model_id='gpt2', model_kwargs=None, pipeline_kwargs=None)

In [29]:
### testing model, not using the harry potter books yet
### answer is not necessarily related to harry potter
query = "Give me 5 examples of cool potions and explain what they do"
print(llm(query))

. Unterscheidung between different types of potions, such as healing, damage dealing, buffs, debuffs, etc.
Sure! Here are five examples of cool potions in the world of fantasy:
1. Healing Potion: This is a classic example of a potion that restores health to a character. When consumed, it can restore a set amount of health points, potentially saving a character from death or at least buying them some time to escape danger. The effects of this potion could be temporary, lasting for a few minutes before wearing off, or permanent, giving the character a steady stream of health regeneration until they consume another potion or die.
2. Damage Dealing Potion: In contrast to the healing potion, this one deals direct damage to enemies. When consumed, it could create a burst of energy that propels a projectile (such as a fireball) towards nearby enemies, dealing significant damage. Alternatively, it could grant the consumer a temporary boost to their attack power, allowing them to land more hits

In [30]:
%%time

loader = DirectoryLoader(CFG.PDFs_path,
                        glob='./*.pdf',
                        loader_cls=PyPDFLoader,
                        show_progress=True,
                        use_multithreading=True)

documents = loader.load()

100%|██████████| 7/7 [02:10<00:00, 18.66s/it]

CPU times: user 2min 9s, sys: 1.64 s, total: 2min 11s
Wall time: 2min 10s


In [31]:
len(documents)

4114

In [33]:
print(documents[3].page_content)

3"HELLO? HELLO? CAN YOU HEAR ME? I -- WANT -- TO -- TALK -- TO --
HARRY-- POTTER!"
Ron was yelling so loudly that Uncle Vernon jumped and held the receiver
a foot away from his ear, staring at it with an expression of mingledfury and alarm.
"WHO IS THIS?" he roared in the direction of the mouthpiece. "WHO ARE
YOU?"
"RON -- WEASLEY!" Ron bellowed back, as though he and Uncle Vernon were
speaking from opposite ends of a football field. "I'M -- A -- FRIEND --OF -- HARRY'S -- FROM -- SCHOOL --"
Uncle Vernon's small eyes swiveled around to Harry, who was rooted to
the spot.
"THERE IS NO HARRY POTTER HERE!" he roared, now holding the receiver
atarm's length, as though frightened it might explode. "I DON'T KNOW WHATSCHOOL YOURE TALKING ABOUT! NEVER CONTACT ME AGAIN!DON'T YOU COME NEARMY FAMILY!"
And he threw the receiver back onto the telephone as if dropping a
poisonous spider.
The fight that had followed had been one of the worst ever."HOW DARE YOU GIVE THIS NUMBER TO PEOPLE LIKE -- PEOPLE 

In [36]:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=CFG.split_chunk_size,
                                                  chunk_overlap=CFG.split_overlap)
    texts = text_splitter.split_documents(documents)
    len(texts)

10519

In [83]:
os.listdir('../input/hp-embeddings-instructor-base-800-0/harry-potter-vectordb-chroma/')

['chroma.sqlite3', '71abcdfe-d9af-49f3-a480-208ac8076ac9']

In [87]:
os.path.exists(CFG.embeddings_path)

True

In [90]:
CFG.embeddings_model_repo

'hkunlp/instructor-base'

In [98]:
CFG.embeddings_path

'/kaggle/working/hp-embeddings-instructor-base-800-0/harry-potter-vectordb-chroma'

In [101]:
%%time
### load vector database
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name=CFG.embeddings_model_repo,
                                                     model_kwargs={"device":"cuda"})

vectordb = Chroma(persist_directory=CFG.embeddings_path,
                 embedding_function=instructor_embeddings,
                 collection_name='hp_books')

load INSTRUCTOR_Transformer
max_seq_length  512
CPU times: user 1.21 s, sys: 145 ms, total: 1.35 s
Wall time: 1.35 s


In [102]:
print("Documents loaded : %d"%(vectordb._collection.count()))

Documents loaded : 10519


#### Generate prompt template

In [105]:
prompt_template = """
Don't try to make up an answer, if you don't know just say that you don't know.
Answer in the same language the question was asked.
Use only the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""


PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

In [142]:
retriever = vectordb.as_retriever(search_kwargs={
    "k" : CFG.k,
    "search_type" : "similarity"
})

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                      chain_type="stuff",
                                      retriever=retriever,
                                      chain_type_kwargs={"prompt":PROMPT},
                                      return_source_documents=True,
                                       verbose=False
                                      )

### Testing

In [143]:
question = "Which are Hagrid's favorite animals?"

In [144]:
#MMR
output = vectordb.max_marginal_relevance_search(question, k=CFG.k)
print(output[0].page_content)

“Well,	so	they	say,”	said	Hagrid.	“Crikey,	I’d	like	a	dragon.”


In [145]:
# similarity search
output1 = vectordb.similarity_search(question, k=CFG.k)
print(output1[0].page_content)

CHAPTER  THIRTEEN 
 198  nothing better than a pet drag on, as Harry, Ron, and Hermione 
knew only too well — he had owned one for a brief period during 
their first year, a vicious Norweg ian Ridgeback by the name of 
Norbert. Hagrid simply loved monstrous creatures, the more 
lethal, the better. 
“Well, at least the skrewts are sma ll,” said Ron as they made their 
way back up to the castle for lunch an hour later. 
“They are now, ” said Hermione in an exasperated voice, “but 
once Hagrid’s found out what they eat, I expect they’ll be six feet 
long.” 
“Well, that won’t matter if they turn out to cure seasickness or 
something, will it?” said Ro n, grinning slyly at her. 
“You know perfectly we ll I only said that to shut Malfoy up,”


In [146]:
def wrap_text_preserve_newlines(text, width=200): # 110
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [147]:
def process_llm_response(llm_response):
    ans = wrap_text_preserve_newlines(llm_response['result'])
    sources_used = ' \n'.join([str(source.metadata['source']) for source in llm_response['source_documents']])
    ans = ans + '\n\nSources: \n' + sources_used
    return ans

In [148]:
def llm_ans(query):
    start = time.time()
    llm_response = qa_chain(query)
    ans = process_llm_response(llm_response)
    end = time.time()

    time_elapsed = int(round(end - start, 0))
    time_elapsed_str = f'\n\nTime elapsed: {time_elapsed} s'
    return ans + time_elapsed_str

# Testing

In [149]:
CFG.model_name

'llama2-7b'

In [150]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [153]:
query = "What are potions?"
print(llm_ans(query))

 Potions are magical liquids used to treat various illnesses or conditions. They can also be used to counteract poisons or to transform something into something else. In the wizarding world, potions
are an important part of medical care and are often used by wizards and witches to maintain their health and well-being.

Sources: 
/kaggle/input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 6 - The Half-Blood Prince.pdf 
/kaggle/input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 5 - The Order of the Phoenix.pdf 
/kaggle/input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 6 - The Half-Blood Prince.pdf

Time elapsed: 9 s


In [154]:
query = "Is Malfoy evil or nice?"
print(llm_ans(query))

 Malfoy is neither evil nor nice.

Sources: 
/kaggle/input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 7 - The Deathly Hallows.pdf 
/kaggle/input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 6 - The Half-Blood Prince.pdf 
/kaggle/input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 7 - The Deathly Hallows.pdf

Time elapsed: 4 s


In [155]:
query = "What are hocruxes?"
print(llm_ans(query))

 Hocruxes are objects in which a piece of a person's soul is stored.

Sources: 
/kaggle/input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 6 - The Half-Blood Prince.pdf 
/kaggle/input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 7 - The Deathly Hallows.pdf 
/kaggle/input/harry-potter-books-in-pdf-1-7/HP books/Harry Potter - Book 7 - The Deathly Hallows.pdf

Time elapsed: 5 s


In [156]:
query = "Give me 5 examples of cool potions and explain what they do"
print(llm_ans(query))

 Sure! Here are five examples of cool potions from the Harry Potter series along with their effects:
1. The Draught of Peace - As its name suggests, this potion is designed to calm anxiety and soothe agitation. However, be careful when brewing this potion as excessive use can lead to drowsiness and
even unconsciousness.
2. The Bubbling Cauldron of Confusion - This potion is used to confuse and disorient opponents, making them more vulnerable to attack. It's often used in battle situations to gain an advantage over
the enemy.
3. The Pixie Pop of Protection - This potion creates a shield of protection around the drinker, deflecting any harmful spells or attacks. It's especially useful during dangerous missions or encounters
with dark magic.
4. The Fizzing Whizbe of Flight - This potion allows the drinker to fly, granting them temporary wings that allow them to soar through the skies. While it may seem like a fun and exciting ability,
it's important to remember that flying without proper 